In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings
import opendatasets as od
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
dataset = 'https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri'
od.download(dataset)
force=True

Skipping, found downloaded files in "./brain-tumor-classification-mri" (use force=True to force download)


In [3]:
for dirname, _, filenames in os.walk('./brain-tumor-classification-mri'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./brain-tumor-classification-mri/Training/no_tumor/image(115).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(142).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(49).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(1).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(284).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(154).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(292).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(103).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(197).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(178).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(73).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(251).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(314).jpg
./brain-tumor-classification-mri/Training/no_tumor/image(24).jpg
./brain-tumor-classification-mri/Training/no_tumor/image (26).jpg
./brain-tumor-c

./brain-tumor-classification-mri/Training/pituitary_tumor/p (413).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (805).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (106).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (556).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (278).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (782).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (628).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (46).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (297).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (444).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (501).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (151).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (50).jpg
./brain-tumor-classification-mri/Training/pituitary_tumor/p (794).jpg
./brain-tumor-classifi

./brain-tumor-classification-mri/Testing/no_tumor/image(49).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(1).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(103).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(73).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(24).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(32).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(65).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(12).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(45).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(53).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(90).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(28).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(86).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(69).jpg
./brain-tumor-classification-mri/Testing/no_tumor/image(87).jpg
./brain-tumor-classification-mri/Testing

In [4]:
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_red = ["#331313", "#582626", '#9E1717', '#D35151', '#E9B4B4']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']

In [5]:
labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']

In [6]:
X_train = []
y_train = []
image_size = 150
for i in labels:
    folderPath = os.path.join(r'./brain-tumor-classification-mri','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        
for i in labels:
    folderPath = os.path.join(r'./brain-tumor-classification-mri','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)
        
X_train = np.array(X_train)
y_train = np.array(y_train)

100%|██████████████████████████████████████████| 74/74 [00:00<00:00, 192.32it/s]


In [7]:
X_train, y_train = shuffle(X_train,y_train, random_state=101)

In [8]:
X_train.shape

(3264, 150, 150, 3)

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.1,random_state=101)

In [10]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [11]:
effnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))

In [12]:
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 150, 150, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 150, 150, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 150, 150, 3)  0           ['normalization[0][0]']      

 block2b_expand_conv (Conv2D)   (None, 38, 38, 144)  3456        ['block2a_project_bn[0][0]']     
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 38, 38, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block2b_expand_activation (Act  (None, 38, 38, 144)  0          ['block2b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block2b_dwconv (DepthwiseConv2  (None, 38, 38, 144)  1296       ['block2b_expand_activation[0][0]
 D)                                                              ']                               
          

                                                                                                  
 block3b_dwconv (DepthwiseConv2  (None, 19, 19, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_bn (BatchNormalization  (None, 19, 19, 240)  960        ['block3b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3b_activation (Activation  (None, 19, 19, 240)  0          ['block3b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3b_s

 block4b_activation (Activation  (None, 10, 10, 480)  0          ['block4b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block4b_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block4b_se_squeeze[0][0]']     
                                                                                                  
 block4b_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block4b_se_reshape[0][0]']     
                                                                                                  
 block4b_s

                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
                                                                                                  
 block5a_se_expand (Conv2D)     (None, 1, 1, 480)    10080       ['block5a_se_reduce[0][0]']      
                                                                                                  
 block5a_se_excite (Multiply)   (None, 10, 10, 480)  0           ['block5a_activation[0][0]',     
                                                                  'block5a_se_expand[0][0]']      
                                                                                                  
 block5a_p

 block5c_project_conv (Conv2D)  (None, 10, 10, 112)  75264       ['block5c_se_excite[0][0]']      
                                                                                                  
 block5c_project_bn (BatchNorma  (None, 10, 10, 112)  448        ['block5c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5c_drop (Dropout)         (None, 10, 10, 112)  0           ['block5c_project_bn[0][0]']     
                                                                                                  
 block5c_add (Add)              (None, 10, 10, 112)  0           ['block5c_drop[0][0]',           
                                                                  'block5b_add[0][0]']            
                                                                                                  
 block6a_e

 block6b_add (Add)              (None, 5, 5, 192)    0           ['block6b_drop[0][0]',           
                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 5, 5, 1152)   221184      ['block6b_add[0][0]']            
                                                                                                  
 block6c_expand_bn (BatchNormal  (None, 5, 5, 1152)  4608        ['block6c_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6c_expand_activation (Act  (None, 5, 5, 1152)  0           ['block6c_expand_bn[0][0]']      
 ivation)                                                                                         
          

 block7a_expand_bn (BatchNormal  (None, 5, 5, 1152)  4608        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 5, 5, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block7a_dwconv (DepthwiseConv2  (None, 5, 5, 1152)  10368       ['block7a_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block7a_bn (BatchNormalization  (None, 5, 5, 1152)  4608        ['block7a_dwconv[0][0]']         
 )        

In [14]:
model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [15]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [16]:
history = model.fit(X_train,y_train,validation_split=0.1, epochs =12, verbose=1, batch_size=32,
                   callbacks=[tensorboard,checkpoint,reduce_lr])

Epoch 1/12
83/83 [==============================] - ETA: 0s - loss: 0.4660 - accuracy: 0.8260
Epoch 1: val_accuracy improved from -inf to 0.82653, saving model to effnet.h5
83/83 [==============================] - 160s 2s/step - loss: 0.4660 - accuracy: 0.8260 - val_loss: 0.7044 - val_accuracy: 0.8265 - lr: 0.0010
Epoch 2/12
83/83 [==============================] - ETA: 0s - loss: 0.2013 - accuracy: 0.9319
Epoch 2: val_accuracy improved from 0.82653 to 0.93197, saving model to effnet.h5
83/83 [==============================] - 151s 2s/step - loss: 0.2013 - accuracy: 0.9319 - val_loss: 0.3004 - val_accuracy: 0.9320 - lr: 0.0010
Epoch 3/12
83/83 [==============================] - ETA: 0s - loss: 0.1449 - accuracy: 0.9493
Epoch 3: val_accuracy did not improve from 0.93197
83/83 [==============================] - 151s 2s/step - loss: 0.1449 - accuracy: 0.9493 - val_loss: 0.3167 - val_accuracy: 0.9082 - lr: 0.0010
Epoch 4/12
83/83 [==============================] - ETA: 0s - loss: 0.0942 - 

In [17]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

11/11 [==============================] - 11s 736ms/step


In [18]:
print(classification_report(y_test_new,pred))

              precision    recall  f1-score   support

           0       0.99      0.91      0.95        93
           1       0.98      1.00      0.99        51
           2       0.93      0.99      0.96        96
           3       1.00      1.00      1.00        87

    accuracy                           0.97       327
   macro avg       0.98      0.98      0.97       327
weighted avg       0.97      0.97      0.97       327



In [19]:
def img_pred(upload):
    for name, file_info in uploader.value.items():
        img = Image.open(io.BytesIO(file_info['content']))
    opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img = cv2.resize(opencvImage,(150,150))
    img = img.reshape(1,150,150,3)
    p = model.predict(img)
    p = np.argmax(p,axis=1)[0]

    if p==0:
        p='Glioma Tumor'
    elif p==1:
        print('The model predicts that there is no tumor')
    elif p==2:
        p='Meningioma Tumor'
    else:
        p='Pituitary Tumor'

    if p!=1:
        print(f'The Model predicts that it is a {p}')

In [20]:
uploader = widgets.FileUpload()
display(uploader)

FileUpload(value={}, description='Upload')

In [21]:
button = widgets.Button(description='Predict')
out = widgets.Output()
def on_button_clicked(_):
    with out:
        clear_output()
        try:
            img_pred(uploader)
            
        except:
            print('No Image Uploaded/Invalid Image File')
button.on_click(on_button_clicked)
widgets.VBox([button,out])